# Korean LLM (Large Language Model) Serving on SageMaker with AWS Large Model Container DLC
---

한국어 LLM 모델 SageMaker 서빙 핸즈온 (No inference code)

- LLM GitHub: https://github.com/nlpai-lab/KULLM
- Hugging Face model hub: https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2
- [AWS Blog: Deploy large models on Amazon SageMaker using DJLServing and DeepSpeed model parallel inference](https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference)

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates')

In [2]:
!pip install -qU boto3 huggingface_hub sagemaker langchain deepspeed 

In [3]:
import os
import sagemaker, boto3, jinja2
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3")  # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment()  # jinja environment to generate model configuration templates

<br>

## 1. Download LLM model and upload it to S3
---

In [4]:
from huggingface_hub import snapshot_download
from pathlib import Path

# - This will download the model into the current directory where ever the jupyter notebook is running
local_model_path = Path("/home/ec2-user/SageMaker/hf_cache")
local_model_path.mkdir(exist_ok=True)

model_id = "nlpai-lab/kullm-polyglot-12.8b-v2"
model_prefix = model_id.split('/')[-1].replace('.', '-')

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
model_download_path = snapshot_download(
    repo_id=model_id,
    cache_dir=local_model_path,
    allow_patterns=allow_patterns,
)

s3_code_prefix = f"ko-llm/{model_prefix}/code"  # folder within bucket where code artifact will go
s3_model_prefix = f"ko-llm/{model_prefix}/model"  # folder where model checkpoint will go
s3_model_artifact = f"s3://{bucket}/{s3_model_prefix}"

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
%%bash
aws configure set default.s3.max_concurrent_requests 100
aws configure set default.s3.max_queue_size 10000
aws configure set default.s3.multipart_threshold 1GB
aws configure set default.s3.multipart_chunksize 64MB

In [6]:
!aws s3 sync {model_download_path} {s3_model_artifact}
print(f"Model uploaded to --- > {s3_model_artifact}")
print(f"We will set option.s3url={s3_model_artifact}")

Model uploaded to --- > s3://sagemaker-us-east-1-143656149352/ko-llm/kullm-polyglot-12-8b-v2/model
We will set option.s3url=s3://sagemaker-us-east-1-143656149352/ko-llm/kullm-polyglot-12-8b-v2/model


<br>

## 2. Model Serving Scripts
---
### Create `serving.properties`

이 설정 파일은 어떤 추론 최적화 라이브러리를 사용할지, 어떤 설정을 사용할지 DJL Serving에 알려주는 설정 파일입니다. 필요에 따라 적절한 구성을 설정할 수 있습니다.

모델이 레이어에 따라 분할되는 파이프라인 병렬화(Pipeline Parallelism)를 사용하는 허깅페이스 Accelerate와 달리, DeepSpeed는 각 레이어(텐서)가 여러 디바이스에 걸쳐 샤딩되는 텐서 병렬화(Tensor Parallelism)를 사용합니다. 파이프라인 병렬 처리 접근 방식에서는 데이터가 각 GPU 장치를 통해 순차적으로 흐르지만, 텐서 병렬 처리는 데이터가 모든 GPU 장치로 전송되어 각 GPU에서 부분적인 결과가 계산됩니다. 그런 다음 All-Gather 연산을 통해 부분 결과를 수집하여 최종 결과를 계산합니다. 따라서, 텐서 병렬화가 일반적으로 더 높은 GPU 활용률과 더 나은 성능을 제공합니다.

- `option.s3url` - 모델 파일의 위치를 지정합니다. 또는`option.model_id` 옵션을 대신 사용하여 허깅페이스 허브에서 모델을 지정할 수 있습니다(예: EleutherAI/gpt-j-6B). 그러면 허브에서 모델이 자동으로 다운로드됩니다. s3url 접근 방식은 자체 환경 내에서 모델 아티팩트를 호스팅할 수 있고 DJL 추론 컨테이너 내에서 최적화된 접근 방식을 활용하여 S3에서 호스팅 인스턴스로 모델을 전송함으로써 더 빠른 모델 배포가 가능합니다.

`serving.properties`의 일반적인 설정법과 자세한 내용은 https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-configuration.html 를 참조하세요.

<img src="../images/TensorShard.png" width="800"/>

In [7]:
src_path = f"src/{model_prefix}"
!rm -rf {src_path}
os.makedirs(src_path, exist_ok=True)

In [8]:
%%writefile {src_path}/serving.properties
engine = FasterTransformer
option.entryPoint = djl_python.fastertransformer
option.s3url = {{s3url}}
option.tensor_parallel_degree = 4
option.dtype = fp16
option.task = text-generation

Writing src/kullm-polyglot-12-8b-v2/serving.properties


### serving.properties의 S3 경로 수정

In [9]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path(f"{src_path}/serving.properties").open().read())
Path(f"{src_path}/serving.properties").open("w").write(template.render(s3url=s3_model_artifact))
!pygmentize {src_path}/serving.properties | cat -n

     1	engine = FasterTransformer
     2	option.entryPoint = djl_python.fastertransformer
     3	option.s3url = s3://sagemaker-us-east-1-143656149352/ko-llm/kullm-polyglot-12-8b-v2/model
     4	option.tensor_parallel_degree = 4
     5	option.dtype = fp16
     6	option.task = text-generation


### Create the Tarball and then upload to S3

In [10]:
!rm -rf model.tar.gz
!tar czvf model.tar.gz -C {src_path} .

./
./serving.properties


In [11]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")
#!rm -rf model.tar.gz

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-143656149352/ko-llm/kullm-polyglot-12-8b-v2/code/model.tar.gz


<br>

## 3. Serve LLM Model on SageMaker

---

### Create SageMaker Model

SageMaker 엔드포인트 생성 매개변수 VolumeSizeInGB를 지정할 때 마운트되는 Amazon EBS(Amazon Elastic Block Store) 볼륨에 /tmp를 매핑하기 때문에 컨테이너는 인스턴스의 `/tmp` 공간에 모델을 다운로드합니다. 이때 s5cmd (https://github.com/peak/s5cmd) 를 활용하므로 대용량 모델을 빠르게 다운로드할 수 있습니다.
볼륨 인스턴스와 함께 미리 빌드되어 제공되는 p4dn과 같은 인스턴스의 경우 컨테이너의 `/tmp`를 계속 활용할 수 있습니다. 

In [12]:
from sagemaker.utils import name_from_base
from sagemaker import image_uris

img_uri = image_uris.retrieve(framework="djl-fastertransformer", region=region, version="0.23.0")
model_name = name_from_base(f"{model_prefix}")
print(f"Image going to be used is ---- > {img_uri}")
print(model_name)

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.23.0-fastertransformer5.3.0-cu118
kullm-polyglot-12-8b-v2-2023-07-24-13-41-56-921


In [13]:
from sagemaker import Model, image_uris, serializers, deserializers
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")
env = {"HUGGINGFACE_HUB_CACHE": "/tmp", "TRANSFORMERS_CACHE": "/tmp"}

model = Model(image_uri=img_uri, model_data=s3_code_artifact, env=env, role=role)

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-143656149352/ko-llm/kullm-polyglot-12-8b-v2/code/model.tar.gz


### Create SageMaker Endpoint

In [14]:
from sagemaker.utils import name_from_base

instance_type = "ml.g5.12xlarge"
endpoint_name = name_from_base(f"{model_prefix}-djl-ft")
print(endpoint_name)

model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=3600,
    wait=False
)

kullm-polyglot-12-8b-v2-djl-ft-2023-07-24-13-42-21-716


엔드포인트가 생성되는 동안 아래의 문서를 같이 확인해 보세요.
- https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html

In [15]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
%%time 
from inference_lib import describe_endpoint, Prompter
describe_endpoint(endpoint_name)         

Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating
Endpoint is  Creating


<br>

## 4. Inference
---

In [17]:
# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer(),
)

In [19]:
%%time
predictor.predict(
    {
        "inputs": ["아마존 웹 서비스에 대해서 자세히 알려줘"],
        "parameters": {"max_seq_len": 200, "temperature": 0.1},
    }
)

CPU times: user 13.7 ms, sys: 4.2 ms, total: 17.9 ms
Wall time: 3.5 s


[{'generated_text': '!A: 아마존 웹 서비스(AWS)는 클라우드 컴퓨팅 서비스를 제공하는 회사입니다. 클라우드 컴퓨팅은 인터넷을 통해 컴퓨팅 리소스를 제공하는 것을 말합니다. AWS는 컴퓨팅 리소스를 제공하고, 이를 통해 기업이 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원합니다.AWS는 전 세계에서 가장 큰 클라우드 컴퓨팅 서비스 제공업체 중 하나이며, 전 세계에서 가장 큰 클라우드 컴퓨팅 회사입니다.AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원하는 서비스를 제공합니다. AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원하는 서비스를 제공합니다.AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할 수 있도록 지원하는 서비스를 제공합니다.AWS는 고객이 AWS를 사용하여 애플리케이션과 서비스를 구축하고 실행할'}]

In [20]:
import json
from inference_lib import KoLLMSageMakerEndpoint

ep = KoLLMSageMakerEndpoint(endpoint_name)

In [21]:
params = {
    "max_seq_len": 512,
    "temperature": 0.2,
    "top_k": 0,
    "top_p": 0.9,
}
instruction = ""
input_text = "아마존 웹 서비스에 대해서 자세히 알려줘"
payload = ep.get_payload(instruction, input_text, params)

In [22]:
%%time
predictor.predict(payload)

CPU times: user 2.83 ms, sys: 0 ns, total: 2.83 ms
Wall time: 8.4 s


[{'generated_text': 'AWS는 Amazon Web Services의 약자입니다. 클라우드 컴퓨팅 서비스를 제공하는 선도 기술 회사입니다. AWS는 전 세계 고객에게 인프라, 애플리케이션, 서비스를 제공하는 데 사용되는 클라우드 컴퓨팅 플랫폼입니다. AWS는 고객이 온 컴퓨팅을 통해 애플리케이션과 애플리케이션을 저장하고 확장할 수 있는 서비스형 인프라(IaaS) 및 서비스형 플랫폼(PaaS)을 제공합니다. 또한 AWS는 고객이 클라우드 기반 애플리케이션을 빌, 테스트 및 관리할 수 있는 서비스형 소프트웨어(SaaS)도 제공합니다. AWS는 고객이 클라우드 컴퓨팅을 최대한 활용할 수 있도록 지원하는 데 중점을 두고 있으며, 고객이 클라우드 기반 애플리케이션을 최대한 활용할 수 있도록 지원하는 데 중점을 두고 있습니다.<|endoftext|>[머니투데이 최광 기자][[the300]"국민의당, \'새정치민주연합\'으로 당명 변경]]본문 이미지 영역 새정치민주연합 문재인 대표가 13일 오후 서울 여의도 국회에서 열린 당무위원회의에 참석해 생각에 잠겨 있다. 2015.12.21/뉴스1 문재인 새정치민주연합 대표는 21일 당무국민의당이 \'새정치민주연합\'으로 당명을 변경하는 것을 환영한다"고 밝혔다.문 대표는 이날 국회에서 열린 당무위원회의에서 "국민의당이 \'새정치민주연합\'으로 당명을 변경하는 것은 국민의당이 새정치민주연합에서 소중과 가치를 존중하고 계승하겠다는 의지를 천명한 것으로 평가한다"며 이같이 말했다.문 대표는 "국민의당이 새정치민주연합으로 당명을 변경하 것은 야권의 분열을 극복하고 통합의 이루겠다는 의지를 천명한 것으로 평가한다"며 "새의당이 새정치민주연합의 당명 변경을 존중하고 수용한 것을 환영한다"고 덧붙였다.[내 삶을 바꾸는 정치뉴스 \'the 300\' 바로가기][\'스페셜 걸\' 포토][손안의 경제뉴스 머니투데이 모바일웹][\'취업의 모든 것\' 잡드림 ]최광기자 hollim324@mt.co.kr<|endoftext|>안녕\'m o

In [23]:
%store endpoint_name

Stored 'endpoint_name' (str)


<br>

## 5. Clean Up
---

In [24]:
!rm -rf src

In [25]:
# Delete the SageMaker endpoint
predictor.delete_model()
predictor.delete_endpoint()

<br>

# References
---

- Model 정보
    - kullm-polyglot-5.8b-v2
        - This model is a parameter-efficient fine-tuned version of EleutherAI/polyglot-ko-5.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2        
    - kullm-polyglot-12.8b-v2
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KULLM v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b      
- 코드
    - [Boto3](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/pytorch_deploy_large_GPT_model/GPT-J-6B-model-parallel-inference-DJL.ipynb)
    - [Python SDK](https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb)
    - [Kor LLM on SageMaker](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker)
    - [AWS Generative AI Workshop for Korean language](https://github.com/aws-samples/aws-ai-ml-workshop-kr/tree/master/genai)